# Lambda Code to stop a notebook instance

In [1]:
import boto3
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)


Let's check the SageMaker API and see how can we find the instances that are running in the account

In [10]:
client = boto3.client('sagemaker')
    
candidates = client.list_notebook_instances()
candidates

{'NotebookInstances': [{'NotebookInstanceName': 'fastai-with-efs',
   'NotebookInstanceArn': 'arn:aws:sagemaker:eu-west-1:217431963147:notebook-instance/fastai-with-efs',
   'NotebookInstanceStatus': 'InService',
   'Url': 'fastai-with-efs.notebook.eu-west-1.sagemaker.aws',
   'InstanceType': 'ml.m4.2xlarge',
   'CreationTime': datetime.datetime(2018, 8, 13, 13, 13, 13, 863000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2018, 8, 24, 10, 46, 41, 550000, tzinfo=tzlocal()),
   'NotebookInstanceLifecycleConfigName': 'MountEFS'}],
 'ResponseMetadata': {'RequestId': '3f8ecced-cad7-40ef-8659-42449daf6de4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3f8ecced-cad7-40ef-8659-42449daf6de4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '406',
   'date': 'Fri, 24 Aug 2018 15:31:18 GMT'},
  'RetryAttempts': 0}}

We want to find the name of the instance. It is used to send the stop and start commands

In [11]:
for candidate in candidates['NotebookInstances']:
    candidateName = candidate['NotebookInstanceName']

candidateName

'fastai-with-efs'

To discover the tags of the notebook instances we need to know the ARN of each

In [12]:
instanceArn = candidate['NotebookInstanceArn']
instanceTags = client.list_tags(ResourceArn=instanceArn)
instanceTags['Tags']

[{'Key': 'Demo', 'Value': 'fastai'}, {'Key': 'InDuty', 'Value': 'Yes'}]

We will look for a tag with key of 'InDuty' and a value of 'Yes'

In [33]:
for tag in instanceTags['Tags']:
    if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
        print("Found it")

Now we can connect it all together

In [35]:
client = boto3.client('sagemaker')
    
candidates = client.list_notebook_instances()

for candidate in candidates['NotebookInstances']:
    candidateName = candidate['NotebookInstanceName']
    logger.info('instance {0} is in Service'.format(candidateName))
    instanceArn = candidate['NotebookInstanceArn']
    instanceTags = client.list_tags(ResourceArn=instanceArn)
    for tag in instanceTags['Tags']:
        if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
            logger.info('instance {0} is in duty'.format(candidateName))
    #        response = client.stop_notebook_instance(NotebookInstanceName=candidateName)
            logger.info('instance {0} was stopped'.format(candidateName))
        else:
            logger.info('Not in duty'.format(candidateName))


# Creating a lambda function

Using writefile magic command, we will generate the python file to be uploaded to AWS Lambda. Also if you want to append to file you must use -a parameter.

In [2]:
%%writefile lambda_function.py
import boto3
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

client = boto3.client('sagemaker')

def lambda_handler(event, context):
    
    candidates = client.list_notebook_instances()
    
    for candidate in candidates['NotebookInstances']:
        candidateName = candidate['NotebookInstanceName']
        logger.info('instance {0} is in Service'.format(candidateName))
        instanceArn = candidate['NotebookInstanceArn']
        instanceTags = client.list_tags(ResourceArn=instanceArn)
        for tag in instanceTags['Tags']:
            if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
                logger.info('instance {0} is in duty'.format(candidateName))
                if (event['event'] == "On"):
                    response = client.start_notebook_instance(NotebookInstanceName=candidateName)
                    logger.info('instance {0} was started'.format(candidateName))
                elif (event['event'] == "Off"):
                    response = client.stop_notebook_instance(NotebookInstanceName=candidateName)
                    logger.info('instance {0} was stopped'.format(candidateName))
            else:
                logger.info('Not in duty'.format(candidateName))

    return "Done"


Writing lambda_function.py


Adding the lambda function into a zip file (AWS Lambda requires a zip file)

In [4]:
!zip lambda.zip lambda_function.py

  adding: lambda_function.py (deflated 68%)


We can create the bucket and copy the file to it using the awscli or the python SDK. I'll use the aws cli in this example.

In [6]:
!aws s3 mb s3://terraform-serverless-repository --region=eu-west-1

make_bucket: terraform-serverless-repository


Uploading the zip file to the bucket

In [8]:
!aws s3 cp lambda.zip s3://terraform-serverless-repository/v1.0.0/

upload: ./lambda.zip to s3://terraform-serverless-repository/v1.0.0/lambda.zip


With the source code artifact built and uploaded to S3, we can now write our Terraform configuration to deploy it. In a new directory, create a file named lambda.tf containing the following configuration:

In [25]:
%%writefile lambda.tf
provider "aws" {
  region = "us-east-1"
}

resource "aws_lambda_function" "examlpe" {
  function_name = "StopStartSageMakerNotebookInstances"

  # The bucket name as created earlier with "aws s3api create-bucket"
  s3_bucket = "terraform-serverless-repository"
  s3_key    = "v1.0.0/lambda.zip"

  # "main" is the filename within the zip file (main.js) and "handler"
  # is the name of the property under which the handler function was
  # exported in that file.
  handler = "lambda_function.lambda_handler"
  runtime = "Python3.6"

  role = "${aws_iam_role.lambda_exec.arn}"
}



Overwriting lambda.tf


# Creating IAM Role

In [26]:
%%writefile -a lambda.tf
# IAM role which dictates what other AWS services the Lambda function
# may access.
resource "aws_iam_role" "lambda_exec" {
  name = "serverless_example_lambda"

  assume_role_policy = <<EOF
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CloudWatchLogs0",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:eu-west-1:217431963147:log-group:/aws/lambda/*NotebookInstance:*"
        },
        {
            "Sid": "InstanceControl",
            "Effect": "Allow",
            "Action": [
                "sagemaker:ListTags",
                "ec2:ModifyNetworkInterfaceAttribute",
                "sagemaker:DescribeNotebookInstance",
                "ec2:DeleteNetworkInterface",
                "ec2:DescribeSecurityGroups",
                "ec2:CreateNetworkInterface",
                "ec2:DescribeInternetGateways",
                "ec2:DescribeNetworkInterfaces",
                "ec2:DescribeAvailabilityZones",
                "sagemaker:StopNotebookInstance",
                "ec2:DescribeVpcs",
                "sagemaker:StartNotebookInstance",
                "ec2:AttachNetworkInterface",
                "ec2:DescribeSubnets",
                "sagemaker:ListNotebookInstances"
            ],
            "Resource": "*"
        },
        {
            "Sid": "CloudWatchLogs1",
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:eu-west-1:217431963147:*"
        }
    ]
}
EOF
}

Appending to lambda.tf


## Terraform installation

In [21]:
!wget https://releases.hashicorp.com/terraform/0.11.8/terraform_0.11.8_linux_amd64.zip

--2018-08-24 20:39:29--  https://releases.hashicorp.com/terraform/0.11.8/terraform_0.11.8_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 151.101.17.183, 2a04:4e42:4::439
Connecting to releases.hashicorp.com (releases.hashicorp.com)|151.101.17.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17871447 (17M) [application/zip]
Saving to: ‘terraform_0.11.8_linux_amd64.zip’

terraform_0.11.8_li 100%[===================>]  17.04M  28.3MB/s    in 0.6s    

2018-08-24 20:39:30 (28.3 MB/s) - ‘terraform_0.11.8_linux_amd64.zip’ saved [17871447/17871447]



In [22]:
!unzip terraform_0.11.8_linux_amd64.zip

Archive:  terraform_0.11.8_linux_amd64.zip
  inflating: terraform               


Before you can work with a new configuration directory, it must be initialized using terraform init, which in this case will install the AWS provider:

In [24]:
!./terraform init



Initializing provider plugins...
- Checking for available provider plugins on https://releases.hashicorp.com...
- Downloading plugin for provider "aws" (1.33.0)...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corresponding provider blocks in configuration, with the constraint strings
suggested below.

* provider.aws: version = "~> 1.33"

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
